# Dimensionality Reduction

In this notebook, I will propose and emply two dimensionality reduction tecniques to my data

## Import packages

In [ ]:
# import packages - general
import numpy as np
import pandas as pd

# import packages - geospatial
import xarray as xr

# import packages - plotting
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature